#Installations

In [ ]:
pip install transformers

#Imports

In [57]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModel, ElectraTokenizer, ElectraModel
import torch
import pandas as pd
import numpy as np
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

#Functions

In [2]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

## ARABERTV02


In [3]:
# Fonction pour générer des représentations vectorielles avec BERT
def get_bert_embeddings(text,model,tokenizer):

    # Tokenize the text and obtain input tensors
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass to get hidden states (word embeddings)
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings for each token (word) excluding [CLS] and [SEP]
    word_embeddings = outputs.last_hidden_state[:, 1:-1, :].numpy()

    return word_embeddings


## ARAELECTRA


In [4]:
 # Fonction pour générer des représentations vectorielles avec ELECTRA

def get_araelectra_embeddings(text,model, tokenizer):


        # Tokenize the text and obtain input tensors
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

        # Forward pass to get hidden states (word embeddings)
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings for each token (word) excluding [CLS] and [SEP]
        word_embeddings = outputs.last_hidden_state[:, 1:-1, :].numpy()

        return word_embeddings


##ARAGPT2

In [5]:
def generate_gpt2_vectors(text, model, tokenizer):
    # Tokenize the text and convert to tensors
    input_ids = tokenizer.encode(text, return_tensors='pt')

    # Pass the tokens through the GPT-2 model
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract the embeddings for the last layer
    last_hidden_states = outputs.last_hidden_state

    # Return the embeddings for the entire sequence
    return last_hidden_states.squeeze().numpy()


##FONCTIONS DE CALCULE DE SIMILARITE

###Cosine similarity

In [6]:
# Function to calculate cosine similarity between word embeddings
def calculate_cosine_similarity(embeddings):
    # Reshape the embeddings to 2D array (words x embedding dimensions)
    embeddings_2d = embeddings.reshape(embeddings.shape[1], -1)

    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(embeddings_2d)

    return similarity_matrix

In [7]:
def aggregate_word_similarity(similarity_matrix):
    # Calculate the average cosine similarity for the entire matrix
    average_similarity = np.mean(similarity_matrix)

    return average_similarity

In [8]:
# Function to calculate cosine similarity between words
def calculate_cosine_similarity_word(vectors1, vectors2):
    min_dim = min(vectors1.shape[0], vectors2.shape[0])
    vectors1 = vectors1[:min_dim]
    vectors2 = vectors2[:min_dim]

    # Flatten the vectors to calculate cosine similarity between words
    vectors1_flat = vectors1.reshape(1, -1)
    vectors2_flat = vectors2.reshape(1, -1)

    return cosine_similarity(vectors1_flat, vectors2_flat)[0][0]

#Data

In [9]:
Data = [
    {"phrase": "المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاقهم الثقافية.", "note": 5},
    {"phrase": "الطبيب يستخدم التكنولوجيا الحديثة في عمليات الجراحة لتحسين نتائج العلاج.", "note": 4},
    {"phrase": "الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق العمل الدولي.", "note": 4},
    {"phrase": "الفلاح يزرع الحبوب في الربيع ويحصدها في فصل الخريف.", "note": 3},
    {"phrase": "العالِم يبحث عن علاج فعّال للأمراض المزمنة من خلال الأبحاث العلمية.", "note": 5},
    {"phrase": "الفنان يستخدم الألوان والأشكال للتعبير عن رؤيته الفنية الفريدة.", "note": 5},
    {"phrase": "المهندس يعمل على تصميم جسر مستدام للتقليل من تأثيرات التغير المناخي.", "note": 4},
    {"phrase": "الصحفي يكتب مقالًا حول التطورات الاقتصادية في العالم النامي.", "note": 3},
    {"phrase": "الرياضي يتدرب بانتظام لتحسين أدائه في البطولات القادمة.", "note": 4},
    {"phrase": "السفينة تطير في السماء والسمك يسبح في الفضاء الخارجي.", "note": 1},
    {"phrase": "القمر يتألق بالضوء في النهار والشمس تشرق في الليل.", "note": 2},
    {"phrase": "الشجرة تتكلم بلغة الطيور وتشارك في النقاشات الفلسفية.", "note": 1},
    {"phrase": "البحر يسير على الأرض والجبال تطفو في المحيط.", "note": 2},
    {"phrase": "النجم يسافر في الزمن للعودة إلى الماضي وتغيير التاريخ.", "note": 1},
    {"phrase": "الأسد يرتدي نظارات شمسية ويقود سيارة في السافانا.", "note": 1},
    {"phrase": "السماء تمطر بالحليب والأرض تتساقط بالحلوى.", "note": 1},
    {"phrase": "السفراء الفضائيون يأتون إلى الأرض لتبادل الوصفات الطهي.", "note": 1},
    {"phrase": "القمر يبتسم ويغني أغاني الحب في الفضاء.", "note": 1}
]

df = pd.DataFrame(Data)

In [10]:
df

,phrase,note
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,5
1,الطبيب يستخدم التكنولوجيا الحديثة في عمليات ال...,4
2,الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق...,4
3,الفلاح يزرع الحبوب في الربيع ويحصدها في فصل ال...,3
4,العالِم يبحث عن علاج فعّال للأمراض المزمنة من ...,5
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,5
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,4
7,الصحفي يكتب مقالًا حول التطورات الاقتصادية في ...,3
8,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,4
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,1


#Prepreocessing

In [11]:
df['phrase_cleaned'] = df['phrase'].apply(clean_text)

In [12]:
df = df[['phrase', 'phrase_cleaned', 'note']]

In [13]:
df

,phrase,phrase_cleaned,note
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,5
1,الطبيب يستخدم التكنولوجيا الحديثة في عمليات ال...,الطبيب يستخدم التكنولوجيا الحديثة في عمليات ال...,4
2,الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق...,الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق...,4
3,الفلاح يزرع الحبوب في الربيع ويحصدها في فصل ال...,الفلاح يزرع الحبوب في الربيع ويحصدها في فصل ال...,3
4,العالِم يبحث عن علاج فعّال للأمراض المزمنة من ...,العالم يبحث عن علاج فعال للأمراض المزمنة من خل...,5
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,5
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,4
7,الصحفي يكتب مقالًا حول التطورات الاقتصادية في ...,الصحفي يكتب مقالا حول التطورات الاقتصادية في ا...,3
8,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,4
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,1


#Pre-Trained Models

## ARABERTV02


In [14]:
model_name_bert = "aubmindlab/bert-base-arabertv02"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
model_bert = AutoModel.from_pretrained(model_name_bert)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [15]:
# Apply the function to the 'phrase' column and create a new column 'bert_embeddings'
df['bert_embeddings'] = df['phrase_cleaned'].apply(lambda x: get_bert_embeddings(x, model_bert, tokenizer_bert))

<ipython-input-15-59ee8a701603>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bert_embeddings'] = df['phrase_cleaned'].apply(lambda x: get_bert_embeddings(x, model_bert, tokenizer_bert))


In [16]:
df[['phrase_cleaned', 'note', 'bert_embeddings']]

,phrase_cleaned,note,bert_embeddings
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,5,"[[[-1.032975, 1.0678422, 0.47265348, -0.895028..."
1,الطبيب يستخدم التكنولوجيا الحديثة في عمليات ال...,4,"[[[-0.59730935, 1.2014942, -0.069083676, -1.00..."
2,الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق...,4,"[[[-1.5562198, 0.9770045, -0.08045507, -0.0834..."
3,الفلاح يزرع الحبوب في الربيع ويحصدها في فصل ال...,3,"[[[0.06730243, 0.44943416, 0.018147618, -1.351..."
4,العالم يبحث عن علاج فعال للأمراض المزمنة من خل...,5,"[[[-0.4197267, 1.1680038, 0.5660643, -0.260565..."
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,5,"[[[-0.58448905, 0.8276985, -0.290823, 0.513876..."
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,4,"[[[0.008138003, 1.3471119, -0.053471096, -0.59..."
7,الصحفي يكتب مقالا حول التطورات الاقتصادية في ا...,3,"[[[0.3941028, 1.4700148, -0.8629787, 0.2190544..."
8,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,4,"[[[-0.66311765, 1.1386726, -0.042917322, 2.235..."
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,1,"[[[-0.3062546, 0.3541972, 0.30680835, 0.186084..."


##ARAELECTRA

In [17]:
model_name_electra = "aubmindlab/araelectra-base-generator"
tokenizer_electra = ElectraTokenizer.from_pretrained(model_name_electra)
model_electra = AutoModel.from_pretrained(model_name_electra)

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/238M [00:00<?, ?B/s]

In [18]:
# Apply the function to the 'phrase' column and create a new column 'bert_embeddings'
df['alectra_embeddings'] = df['phrase_cleaned'].apply(lambda x: get_araelectra_embeddings(x, model_electra, tokenizer_electra))


<ipython-input-18-aaa450540780>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alectra_embeddings'] = df['phrase_cleaned'].apply(lambda x: get_araelectra_embeddings(x, model_electra, tokenizer_electra))


In [19]:
df[['phrase_cleaned', 'note', 'alectra_embeddings']]

,phrase_cleaned,note,alectra_embeddings
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,5,"[[[-0.04132988, -0.66178715, 0.21291986, 0.468..."
1,الطبيب يستخدم التكنولوجيا الحديثة في عمليات ال...,4,"[[[-0.389818, -0.93290824, -0.08991882, 0.2594..."
2,الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق...,4,"[[[-0.20971271, -0.22505426, 0.17081855, 0.624..."
3,الفلاح يزرع الحبوب في الربيع ويحصدها في فصل ال...,3,"[[[-0.87272865, -0.15812613, 0.15181029, 1.306..."
4,العالم يبحث عن علاج فعال للأمراض المزمنة من خل...,5,"[[[-0.52659756, -0.75845176, 0.51208144, 0.290..."
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,5,"[[[-0.17771685, -0.17310181, 0.67117757, 0.527..."
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,4,"[[[-1.0876894, 0.67977417, 0.021402285, 0.4817..."
7,الصحفي يكتب مقالا حول التطورات الاقتصادية في ا...,3,"[[[-0.41201264, 0.100352906, 0.8722129, 0.2216..."
8,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,4,"[[[-0.07330833, -0.21832684, 0.68965966, 0.236..."
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,1,"[[[-0.029132005, -0.07445603, 0.28978667, 0.79..."


##ARAGPT2

In [20]:
model_name_gpt = "aubmindlab/aragpt2-base"
tokenizer_gpt = AutoTokenizer.from_pretrained(model_name_gpt)
model_gpt = AutoModel.from_pretrained(model_name_gpt)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

In [21]:
df['gpt2_vectors'] = df['phrase_cleaned'].apply(lambda x: generate_gpt2_vectors(x, model_gpt, tokenizer_gpt))

<ipython-input-21-6f68fa411ee5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gpt2_vectors'] = df['phrase_cleaned'].apply(lambda x: generate_gpt2_vectors(x, model_gpt, tokenizer_gpt))


In [22]:
df[['phrase_cleaned', 'note', 'gpt2_vectors']]

,phrase_cleaned,note,gpt2_vectors
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,5,"[[-0.6412745, 0.041314203, -0.40571848, 0.2575..."
1,الطبيب يستخدم التكنولوجيا الحديثة في عمليات ال...,4,"[[-0.77790475, 0.012438345, -0.22074924, 0.094..."
2,الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق...,4,"[[-0.77790475, 0.012438345, -0.22074924, 0.094..."
3,الفلاح يزرع الحبوب في الربيع ويحصدها في فصل ال...,3,"[[0.008000887, -0.39085618, -0.6043445, -0.068..."
4,العالم يبحث عن علاج فعال للأمراض المزمنة من خل...,5,"[[-0.8255977, -0.20569289, -0.43115953, 0.4093..."
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,5,"[[0.008000887, -0.39085618, -0.6043445, -0.068..."
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,4,"[[-0.4731263, -0.5087742, -0.48084402, -0.1673..."
7,الصحفي يكتب مقالا حول التطورات الاقتصادية في ا...,3,"[[-0.5231671, -0.72551244, -0.09326597, 0.6016..."
8,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,4,"[[-0.9985046, -0.7375723, 0.10928986, -0.33582..."
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,1,"[[-0.37186137, -0.02628268, -0.3723074, 0.0549..."


#Performances

##ARABERTV02

In [23]:
# Apply the function to the 'bert_word_embeddings' column
df['cosine_similarity_matrix_bert'] = df['bert_embeddings'].apply(calculate_cosine_similarity)
# Apply the function to the 'cosine_similarity_matrix' column
df['average_similarity_bert'] = df['cosine_similarity_matrix_bert'].apply(aggregate_word_similarity)

<ipython-input-23-e003b7f8740c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cosine_similarity_matrix_bert'] = df['bert_embeddings'].apply(calculate_cosine_similarity)
<ipython-input-23-e003b7f8740c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['average_similarity_bert'] = df['cosine_similarity_matrix_bert'].apply(aggregate_word_similarity)


In [24]:
df['average_similarity_bert']

0     0.488447
1     0.462065
2     0.401274
3     0.542511
4     0.501247
5     0.528249
6     0.422448
7     0.434931
8     0.498113
9     0.554978
10    0.564171
11    0.473856
12    0.495162
13    0.460868
14    0.388822
15    0.552911
16    0.387887
17    0.493527
Name: average_similarity_bert, dtype: float32

##ARAELECTRA

In [25]:
# Apply the function to the 'bert_word_embeddings' column
df['cosine_similarity_matrix_elect'] = df['alectra_embeddings'].apply(calculate_cosine_similarity)
# Apply the function to the 'cosine_similarity_matrix' column
df['average_similarity_elect'] = df['cosine_similarity_matrix_elect'].apply(aggregate_word_similarity)

<ipython-input-25-ccfd7e736666>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cosine_similarity_matrix_elect'] = df['alectra_embeddings'].apply(calculate_cosine_similarity)
<ipython-input-25-ccfd7e736666>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['average_similarity_elect'] = df['cosine_similarity_matrix_elect'].apply(aggregate_word_similarity)


In [26]:
df['average_similarity_elect']

0     0.512724
1     0.577637
2     0.529908
3     0.510415
4     0.533629
5     0.549701
6     0.492707
7     0.532857
8     0.522944
9     0.557177
10    0.530265
11    0.533295
12    0.561317
13    0.597849
14    0.410895
15    0.444032
16    0.481735
17    0.539096
Name: average_similarity_elect, dtype: float32

## ARAGPT 2

In [27]:
# Assuming df is your original DataFrame
new_df = pd.DataFrame(index=df.index, columns=df.index)

# Fill the new DataFrame with cosine similarity scores between words
for i in df.index:
    for j in df.index:
        new_df.loc[i, j] = calculate_cosine_similarity_word(df['gpt2_vectors'][i], df['gpt2_vectors'][j])

# Rename the columns and index for clarity
new_df.columns = [f'word_{i}' for i in new_df.columns]
new_df.index = [f'word_{i}' for i in new_df.index]

# Display the new DataFrame
new_df


,word_0,word_1,word_2,word_3,word_4,word_5,word_6,word_7,word_8,word_9,word_10,word_11,word_12,word_13,word_14,word_15,word_16,word_17
word_0,1.0,0.283899,0.407426,0.193849,0.248979,0.276114,0.225978,0.198142,0.184144,0.16837,0.144133,0.229177,0.165322,0.221175,0.1887,0.121597,0.284911,0.251291
word_1,0.283899,1.000001,0.351035,0.190518,0.364833,0.366125,0.310735,0.205721,0.159682,0.159285,0.16161,0.28721,0.163287,0.274147,0.192783,0.098169,0.199937,0.172008
word_2,0.407426,0.351035,1.0,0.202451,0.227083,0.292445,0.226867,0.227484,0.277977,0.170983,0.161652,0.255152,0.189962,0.258623,0.203123,0.120437,0.206831,0.235785
word_3,0.193849,0.190518,0.202451,1.0,0.225411,0.261396,0.220653,0.13884,0.117061,0.165617,0.270306,0.219924,0.211455,0.19052,0.15629,0.21757,0.176556,0.201726
word_4,0.248979,0.364833,0.227083,0.225411,1.0,0.234506,0.324813,0.228524,0.139314,0.195456,0.185319,0.238334,0.138169,0.283648,0.163021,0.165547,0.177768,0.216847
word_5,0.276114,0.366125,0.292445,0.261396,0.234506,1.0,0.247168,0.193749,0.145675,0.1754,0.290602,0.391379,0.242248,0.368787,0.221044,0.203567,0.225508,0.258332
word_6,0.225978,0.310735,0.226867,0.220653,0.324813,0.247168,1.0,0.179555,0.127777,0.212659,0.183593,0.194023,0.161615,0.250151,0.165128,0.140157,0.150787,0.176363
word_7,0.198142,0.205721,0.227484,0.13884,0.228524,0.193749,0.179555,1.0,0.170669,0.152254,0.151674,0.21166,0.141231,0.223976,0.153546,0.142252,0.190995,0.193956
word_8,0.184144,0.159682,0.277977,0.117061,0.139314,0.145675,0.127777,0.170669,1.0,0.092757,0.130735,0.131745,0.124781,0.153662,0.179839,0.097846,0.161541,0.169338
word_9,0.16837,0.159285,0.170983,0.165617,0.195456,0.1754,0.212659,0.152254,0.092757,1.0,0.193018,0.253491,0.245304,0.288295,0.179714,0.220548,0.198028,0.212715


In [28]:
# Convert similarity scores in the new DataFrame to numeric values
new_df = new_df.apply(pd.to_numeric, errors='coerce')

# Calculate the average similarity score for each phrase
new_df['average_similarity_gpt'] = new_df.mean(axis=1)

# Display the updated DataFrame with the average similarity scores
print(f"Mesure globale de cohérence intra-phrastique (moyenne) : {new_df[['average_similarity_gpt']]}")


Mesure globale de cohérence intra-phrastique (moyenne) :          average_similarity_gpt
word_0                 0.266289
word_1                 0.274499
word_2                 0.278629
word_3                 0.242230
word_4                 0.264310
word_5                 0.299669
word_6                 0.249890
word_7                 0.228013
word_8                 0.198030
word_9                 0.237994
word_10                0.253971
word_11                0.289171
word_12                0.237349
word_13                0.287631
word_14                0.233757
word_15                0.225431
word_16                0.247979
word_17                0.271800


In [29]:
# Calculate Pearson correlation
pearson_corr, _ = pearsonr(new_df['average_similarity_gpt'], df['note'])
print(f"Pearson Correlation: {pearson_corr}")

# Calculate Spearman correlation
spearman_corr, _ = spearmanr(new_df['average_similarity_gpt'], df['note'])
print(f"Spearman Correlation: {spearman_corr}")


Pearson Correlation: 0.15489388045356434
Spearman Correlation: 0.20486853281090003


In [34]:
new_df['average_similarity_gpt']

word_0     0.266289
word_1     0.274499
word_2     0.278629
word_3     0.242230
word_4     0.264310
word_5     0.299669
word_6     0.249890
word_7     0.228013
word_8     0.198030
word_9     0.237994
word_10    0.253971
word_11    0.289171
word_12    0.237349
word_13    0.287631
word_14    0.233757
word_15    0.225431
word_16    0.247979
word_17    0.271800
Name: average_similarity_gpt, dtype: float64

In [38]:
df['average_similarity_gpt'] = new_df['average_similarity_gpt'].reset_index(drop=True).copy()

df['average_similarity_gpt']


<ipython-input-38-81bb6f67e9c4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['average_similarity_gpt'] = new_df['average_similarity_gpt'].reset_index(drop=True).copy()


0     0.266289
1     0.274499
2     0.278629
3     0.242230
4     0.264310
5     0.299669
6     0.249890
7     0.228013
8     0.198030
9     0.237994
10    0.253971
11    0.289171
12    0.237349
13    0.287631
14    0.233757
15    0.225431
16    0.247979
17    0.271800
Name: average_similarity_gpt, dtype: float64

# Calcule Erreur

##GPT

In [70]:
# Scores automatiques de similarité cosinus
automatic_scores = new_df['average_similarity_gpt'].values

# Annotations humaines
human_annotations = df['note'].values

# Ajouter une colonne pour les écarts entre les scores automatiques et les annotations humaines
df['error_gpt'] = np.abs(automatic_scores - human_annotations)


# Initialize an empty list to store MSE values for each row
mse_per_row_gpt = []

# Iterate over rows and calculate MSE for each row
for i in range(len(df)):
    mse_row = mean_squared_error([human_annotations[i]], [automatic_scores[i]])
    mse_per_row_gpt.append(mse_row)

# Convert the list to a NumPy array for easy manipulation
mse_per_row_gpt = np.array(mse_per_row_gpt)

# Create a new column in the DataFrame to store MSE values
df['error_mse_gpt'] = mse_per_row_gpt

# Trier le DataFrame par ordre décroissant d'erreur
sorted_df = df.sort_values(by='error_gpt', ascending=False)


<ipython-input-70-5794b3215c8f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_gpt'] = np.abs(automatic_scores - human_annotations)
<ipython-input-70-5794b3215c8f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_mse_gpt'] = mse_per_row_gpt


###Results

In [68]:
# Afficher les exemples avec les plus grandes erreurs
print("Exemples avec les plus grandes erreurs :")
sorted_df[['phrase_cleaned', 'average_similarity_gpt', 'note', 'error_gpt','error_mse_gpt']].head()

Exemples avec les plus grandes erreurs :


,phrase_cleaned,average_similarity_gpt,note,error_gpt,error_mse_gpt
4,العالم يبحث عن علاج فعال للأمراض المزمنة من خل...,0.264310,5,4.735690,22.426764
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,0.266289,5,4.733711,22.408018
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,0.299669,5,4.700331,22.093111
8,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,0.198030,4,3.801970,14.454974
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,0.249890,4,3.750110,14.063325


In [69]:
# Afficher les exemples avec les plus petites erreurs
print("\nExemples avec les plus petites erreurs :")
sorted_df[['phrase_cleaned', 'average_similarity_gpt', 'note', 'error_gpt','error_mse_gpt']].tail()


Exemples avec les plus petites erreurs :


,phrase_cleaned,average_similarity_gpt,note,error_gpt,error_mse_gpt
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,0.237994,1,0.762006,0.580653
16,السفراء الفضائيون يأتون إلى الأرض لتبادل الوصف...,0.247979,1,0.752021,0.565536
17,القمر يبتسم ويغني أغاني الحب في الفضاء,0.271800,1,0.728200,0.530275
13,النجم يسافر في الزمن للعودة إلى الماضي وتغيير ...,0.287631,1,0.712369,0.507469
11,الشجرة تتكلم بلغة الطيور وتشارك في النقاشات ال...,0.289171,1,0.710829,0.505277


##BERT

In [71]:
# Scores automatiques de similarité cosinus
automatic_scores = df['average_similarity_bert'].values

# Annotations humaines
human_annotations = df['note'].values

# Ajouter une colonne pour les écarts entre les scores automatiques et les annotations humaines
df['error_bert'] = np.abs(automatic_scores - human_annotations)

# Initialize an empty list to store MSE values for each row
mse_per_row_bert = []

# Iterate over rows and calculate MSE for each row
for i in range(len(df)):
    mse_row = mean_squared_error([human_annotations[i]], [automatic_scores[i]])
    mse_per_row_bert.append(mse_row)

# Convert the list to a NumPy array for easy manipulation
mse_per_row_bert = np.array(mse_per_row_bert)

# Create a new column in the DataFrame to store MSE values
df['error_mse_bert'] = mse_per_row_bert


# Trier le DataFrame par ordre décroissant d'erreur
sorted_df = df.sort_values(by='error_bert', ascending=False)

<ipython-input-71-36b296774b3c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_bert'] = np.abs(automatic_scores - human_annotations)
<ipython-input-71-36b296774b3c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_mse_bert'] = mse_per_row_bert


###Results

In [72]:
# Afficher les exemples avec les plus grandes erreurs
print("Exemples avec les plus grandes erreurs :")
sorted_df[['phrase_cleaned', 'average_similarity_bert', 'note', 'error_bert','error_mse_bert']].head()


Exemples avec les plus grandes erreurs :


,phrase_cleaned,average_similarity_bert,note,error_bert,error_mse_bert
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,0.488447,5,4.511553,20.354109
4,العالم يبحث عن علاج فعال للأمراض المزمنة من خل...,0.501247,5,4.498753,20.238774
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,0.528249,5,4.471751,19.996556
2,الطالب يدرس اللغات الأجنبية لزيادة فرصه في سوق...,0.401274,4,3.598726,12.950826
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,0.422448,4,3.577552,12.798877


In [73]:
# Afficher les exemples avec les plus petites erreurs
print("\nExemples avec les plus petites erreurs :")
sorted_df[['phrase_cleaned', 'average_similarity_bert', 'note', 'error_bert','error_mse_bert']].tail()


Exemples avec les plus petites erreurs :


,phrase_cleaned,average_similarity_bert,note,error_bert,error_mse_bert
13,النجم يسافر في الزمن للعودة إلى الماضي وتغيير ...,0.460868,1,0.539132,0.290663
11,الشجرة تتكلم بلغة الطيور وتشارك في النقاشات ال...,0.473856,1,0.526144,0.276828
17,القمر يبتسم ويغني أغاني الحب في الفضاء,0.493527,1,0.506473,0.256515
15,السماء تمطر بالحليب والأرض تتساقط بالحلوى,0.552911,1,0.447089,0.199889
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,0.554978,1,0.445022,0.198044


##ELECTRA

In [75]:
# Scores automatiques de similarité cosinus
automatic_scores = df['average_similarity_elect'].values

# Annotations humaines
human_annotations = df['note'].values

# Ajouter une colonne pour les écarts entre les scores automatiques et les annotations humaines
df['error_elect'] = np.abs(automatic_scores - human_annotations)

# Initialize an empty list to store MSE values for each row
mse_per_row_elctra = []

# Iterate over rows and calculate MSE for each row
for i in range(len(df)):
    mse_row = mean_squared_error([human_annotations[i]], [automatic_scores[i]])
    mse_per_row_elctra.append(mse_row)

# Convert the list to a NumPy array for easy manipulation
mse_per_row_elctra = np.array(mse_per_row_elctra)

# Create a new column in the DataFrame to store MSE values
df['error_mse_elctra'] = mse_per_row_elctra

# Trier le DataFrame par ordre décroissant d'erreur
sorted_df = df.sort_values(by='error_elect', ascending=False)


<ipython-input-75-eac4afe52af6>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_elect'] = np.abs(automatic_scores - human_annotations)
<ipython-input-75-eac4afe52af6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['error_mse_elctra'] = mse_per_row_elctra


###Results

In [76]:
# Afficher les exemples avec les plus grandes erreurs
print("Exemples avec les plus grandes erreurs :")
sorted_df[['phrase_cleaned', 'average_similarity_elect', 'note', 'error_elect','error_mse_elctra']].head()


Exemples avec les plus grandes erreurs :


,phrase_cleaned,average_similarity_elect,note,error_elect,error_mse_elctra
0,المعلم يشجع الطلاب على قراءة الكتب لتوسيع آفاق...,0.512724,5,4.487276,20.135644
4,العالم يبحث عن علاج فعال للأمراض المزمنة من خل...,0.533629,5,4.466371,19.948470
5,الفنان يستخدم الألوان والأشكال للتعبير عن رؤيت...,0.549701,5,4.450299,19.805165
6,المهندس يعمل على تصميم جسر مستدام للتقليل من ت...,0.492707,4,3.507293,12.301102
8,الرياضي يتدرب بانتظام لتحسين أدائه في البطولات...,0.522944,4,3.477056,12.089921


In [77]:
# Afficher les exemples avec les plus petites erreurs
print("\nExemples avec les plus petites erreurs :")
sorted_df[['phrase_cleaned', 'average_similarity_elect', 'note', 'error_elect','error_mse_elctra']].tail()


Exemples avec les plus petites erreurs :


,phrase_cleaned,average_similarity_elect,note,error_elect,error_mse_elctra
16,السفراء الفضائيون يأتون إلى الأرض لتبادل الوصف...,0.481735,1,0.518265,0.268599
11,الشجرة تتكلم بلغة الطيور وتشارك في النقاشات ال...,0.533295,1,0.466705,0.217813
17,القمر يبتسم ويغني أغاني الحب في الفضاء,0.539096,1,0.460904,0.212433
9,السفينة تطير في السماء والسمك يسبح في الفضاء ا...,0.557177,1,0.442823,0.196092
13,النجم يسافر في الزمن للعودة إلى الماضي وتغيير ...,0.597849,1,0.402151,0.161725


##Comparaison

In [78]:
errors = df[['error_bert','error_mse_bert','error_elect','error_mse_elctra','error_gpt','error_mse_gpt']]
errors

,error_bert,error_mse_bert,error_elect,error_mse_elctra,error_gpt,error_mse_gpt
0,4.511553,20.354109,4.487276,20.135644,4.733711,22.408018
1,3.537935,12.516983,3.422363,11.712566,3.725501,13.879357
2,3.598726,12.950826,3.470092,12.041536,3.721371,13.848606
3,2.457489,6.039250,2.489585,6.198032,2.757770,7.605295
4,4.498753,20.238774,4.466371,19.948470,4.735690,22.426764
5,4.471751,19.996556,4.450299,19.805165,4.700331,22.093111
6,3.577552,12.798877,3.507293,12.301102,3.750110,14.063325
7,2.565069,6.579579,2.467143,6.086793,2.771987,7.683914
8,3.501887,12.263215,3.477056,12.089921,3.801970,14.454974
9,0.445022,0.198044,0.442823,0.196092,0.762006,0.580653


In [79]:
# Calculate the average for each column
average_errors = errors.mean()

# Display the average errors
print("Average Errors:")
print(average_errors)

Average Errors:
error_bert          2.186029
error_mse_bert      7.224137
error_elect         2.143434
error_mse_elctra    7.014554
error_gpt           2.411853
error_mse_gpt       8.249734
dtype: float64


#Conclusion

ELECTRA has the best results